In [1]:
import os
from pathlib import Path

import pandas as pd
import re

In [2]:
review_path = Path("data/esd_done/")

df = None
for file in review_path.rglob("*.xlsx"):
    if df is None:
        df = pd.read_excel(file, engine="openpyxl")
    else:
        df = pd.concat([df, pd.read_excel(file, engine="openpyxl")], ignore_index=True)

In [3]:
df['speaker'] = df['filename'].str.split("_").apply(lambda x: x[0][1:])

In [4]:
df.head()

,filename,text,modified text,accept,reject,comment,doubt,speaker
0,F0016_angry_0016_000351,"The nine the eggs , I keep .",NaN,accepted,NaN,NaN,NaN,0016
1,F0016_angry_0016_000352,"I did go , and made many prisoners .",NaN,accepted,NaN,NaN,NaN,0016
2,F0016_angry_0016_000353,That I owe my thanks to you .,NaN,accepted,NaN,NaN,NaN,0016
3,F0016_angry_0016_000354,They went up to the dark mass job had pointed ...,NaN,accepted,NaN,NaN,NaN,0016
4,F0016_angry_0016_000355,Clear than clear water !,NaN,accepted,NaN,NaN,NaN,0016


In [8]:
folders = ["train", "evaluation", "test"]

datapath = Path("data/esd")

for speaker in os.listdir(datapath):
    wav_dir = datapath / speaker / "wav"
    wav_dir.mkdir(exist_ok=True, parents=True)
    
    for emo in ["Angry", "Happy", "Neutral", "Sad", "Surprise"]:
        if not (datapath / speaker / emo ).is_dir():
            continue
        for folder in folders:
            for file in os.listdir(datapath / speaker / emo / folder):
                os.replace(datapath / speaker / emo / folder / file, wav_dir / f"{emo}_{file}")
            if (datapath / speaker / emo / folder).is_dir():
                os.removedirs(datapath / speaker / emo / folder)
        if (datapath / speaker / emo ).is_dir():
            os.removedirs(datapath / speaker / emo)

In [9]:
for speaker in os.listdir(datapath):
    wav_dir = datapath / speaker / "wav"
    for file in os.listdir(wav_dir):
        os.replace(wav_dir / file, wav_dir / file.split("_", 1)[1])


In [219]:
encodings = {
    "0011": "ascii",
    "0012": "utf16",
    "0013": "utf16",
    "0014": "utf16",
    "0015": "ascii",
    "0016": "iso8859",
    "0017": "iso8859",
    "0018": "utf16",
    "0020": "ascii",
}

In [233]:
import tgt

In [247]:
for speaker in os.listdir(datapath):
    for file in os.listdir(f"data/esd_vctk/mfa_outputs/{speaker}/"):
        if os.path.isdir(f"data/esd_vctk/mfa_outputs/{speaker}/{file}"):
            continue
        text_grid = tgt.read_textgrid(f"data/esd_vctk/mfa_outputs/{speaker}/{file}")
        words = text_grid.get_tier_by_name("words")
        text = " ".join([x.text for x in words.intervals])
        with open(datapath / speaker / f"{file.replace('TextGrid', 'txt')}", "w") as f:
            f.write(text + "\n")

In [248]:
rejected = df[df["accept"].isnull()]

In [250]:
for filename, speaker in zip(rejected["filename"], rejected["speaker"]):
    if os.path.isfile(f"data/esd/{speaker}/{filename}.wav"):
        os.remove(f"data/esd/{speaker}/{filename}.wav")
    if os.path.isfile(f"data/esd/{speaker}/{filename}.txt"):
        os.remove(f"data/esd/{speaker}/{filename}.txt")

In [251]:
modified = df[df["accept"] == "modified"]

In [252]:
modified.head()

,filename,text,modified text,accept,reject,comment,doubt,speaker
34,0016_000385,The name really is the agedaged man .,The name really is the adaged man .,modified,NaN,NaN,NaN,0016
85,0016_000436,And tick a tack too .,And a tick a tack too .,modified,NaN,NaN,NaN,0016
381,0016_001435,The name really is the agedaged man .,The name really is the adaged man .,modified,NaN,NaN,NaN,0016
427,0016_001486,And tick a tack too .,And a tick a tack too .,modified,NaN,NaN,NaN,0016
1423,0015_001024,she caught at the brooch .,And she caught at the brooch .,modified,NaN,NaN,NaN,0015


In [254]:
for filename, speaker, orig, modif in zip(modified["filename"], modified["speaker"], modified["text"], modified["modified text"]):
    path = datapath / speaker / f"{filename}.txt"
    with open(path, "w") as f:
        f.write(modif + "\n")

In [5]:
df["speaker"].unique()

array(['0016', '0018', '0015', '0017'], dtype=object)

In [5]:
copy_speakers = list(df["speaker"].unique()) + [s for s in os.listdir("data/esd_vctk/resampled/") if not s.startswith("00")]

In [6]:
exclude_speakers = [s for s in os.listdir("data/esd_vctk/resampled/") if s not in copy_speakers]

In [7]:
exclude_speakers

['0013', '0012', '0011', '0014', '0020']

In [8]:
percentages = [7, 3, 2, 1, 0.5]

In [9]:
from shutil import copy

In [10]:
import json

In [11]:
with open("speaker_emo.json") as f:
    speaker_emo_dict = json.load(f)

In [12]:
def get_files(p, speaker):
    emo_dict = speaker_emo_dict[speaker]
    files = []
    for val in emo_dict.values():
        to_get = int(p * len(val))
        files.extend(val[:to_get])
    return files

In [14]:
ext_dict = {
    "mels": "pkl",
    "mfa_outputs": "TextGrid",
    "resampled": "wav"
}

In [16]:
root_path = Path("data/esd_vctk/")

for p in percentages:
    new_path = Path(f"data/esd_vctk_{p}/")
    for folder in os.listdir(root_path):
        for speaker in copy_speakers:
            speaker_path = new_path / folder / speaker
            speaker_path.mkdir(exist_ok=True, parents=True)
            if (root_path / folder / speaker).is_dir():
                for file in os.listdir(root_path / folder / speaker):
                    copy(root_path / folder / speaker / file, speaker_path / file)
        for speaker in exclude_speakers:
            speaker_path = new_path / folder / speaker
            speaker_path.mkdir(exist_ok=True, parents=True)
            files_copy = get_files(p / 15, speaker)
            for file in files_copy:
                file = f"{file}.{ext_dict[folder]}"
                if (root_path / folder / speaker / file).is_file():
                    copy(root_path / folder / speaker / file, speaker_path / file)